In [799]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [800]:
def show_image(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    fig = plt.figure(frameon=False)
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    

In [802]:
def solution(image_path):
    ############################
    ############################
    image= cv2.imread(image_path)
    # show_image(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)
    _, thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)
    thresh=Closing(thresh)
    # show_image(thresh)
    
    temp = bding_rect(image.copy(),thresh)
    if(temp == None):
        return 'fake'
    img,thresh = temp  

    # show_image(img)
    # show_image(thresh)

    fir,las = non_zero(thresh[-2,:])
    mid = round(img.shape[1]/((las+fir)//2),3) 
    asp_rat = round(img.shape[1]/img.shape[0],3)
    # print("mid",mid)
    # print("white", dist_white(img[-2,fir+1]))

    rav = "north" if dist_white(img[-5,fir+5]) <200 else "south"

    ideal_asp_rat_south = 2.124
    ideal_asp_rat_north = 1.637

    ideal_mid_south = 2.138
    ideal_mid_north = 2.13

    if rav=="south":
        # print("mid", abs(mid-ideal_mid_south))
        # print("asp", abs(asp_rat-ideal_asp_rat_south))
        if(abs(mid-ideal_mid_south)>0.15 or abs(asp_rat-ideal_asp_rat_south)>0.12):
            return"fake"
    else:
        # print("mid", abs(mid-ideal_mid_north))
        # print("asp", abs(asp_rat-ideal_asp_rat_north))
        if(abs(mid-ideal_mid_north)>0.12 or abs(asp_rat-ideal_asp_rat_north)>0.7):
            return"fake"

    
    

    ############################
    ############################
    ## comment the line below before submitting else your code wont be executed##
    # pass
    return "real"



In [803]:
print(solution('test/r1.jpg'))

real


In [804]:
print(solution('test/r2.jpg'))

fake


In [805]:
print(solution('test/r3.jpg'))

fake


In [806]:
print(solution('test/r4.jpg'))

real


In [807]:
print(solution('test/r5.jpg'))

real


In [808]:
print(solution('test/r6.jpg'))

fake


In [809]:
print(solution('test/r13.jpg'))

fake


In [810]:
print(solution('test/r15.jpg'))

real
